In [1]:
from astropy.io import fits
from astropy import wcs
import numpy as np
from astropy.table import Table

In [63]:
def Segmap_add(sources,ID_modifier,segmap_with_sources,segmap_to_adjust,file_out):
    ID=sources
    
    if ID_modifier > 99:
        print('ERROR: ID modifier should be limited to two digits')
        return ;

    ID_modifier=ID_modifier*100000


    ###choose segmentation map and gather image data
    segmap = fits.open(segmap_with_sources)
    image = segmap[0].data

    ###choose which seg map to add info to
    segmap_main = fits.open(segmap_to_adjust)
    image_main = segmap_main[0].data

    ###get translation info to go from pix to wcs, or vice versa
    w=wcs.WCS(segmap[0].header)
    w_main=wcs.WCS(segmap_main[0].header)

    ###copy source into main file
    for i in range(len(ID)):
        print(ID[i])
        indicies=np.argwhere(image==ID[i])  # get locations of source
        indicies=np.transpose(indicies)     # put into form that easier to organize
        r_min=min(indicies[0])              # get indices to create smallest possible matrix of source
        c_min=min(indicies[1])
        r_max=max(indicies[0])+1
        c_max=max(indicies[1])+1
        source=image[r_min:r_max,c_min:c_max]   # get source image
        source[source!=ID[i]]=0                 # delete any source that's not the one of interest
        source[source==ID[i]]=ID_modifier+ID[i]     # add unique id modifier to sources
        edges=w.wcs_pix2world([[c_min,r_max],[c_max,r_min]],1)  # get world coordinates of edges of source image
        area=w_main.wcs_world2pix(edges,1)      # find location where source will be added
        rowdiff=np.absolute(r_max-r_min)        # make sure area in which the source will be added is the same size
        coldiff=np.absolute(c_max-c_min)
        area = np.array(area).astype(int)
        print(rowdiff,coldiff)
        print(area)
        k=image_main[area[1][1]:area[1][1]+rowdiff,area[0][0]:area[0][0]+coldiff]
        for i in range(len(k)):                 # add source into segmentation map
            for ii in range(len(k[0])):
                if source[i][ii]!=0:
                    k[i][ii]=source[i][ii]

    fits.writeto(file_out,image_main,header=segmap_main[0].header)


In [2]:
N_cat = Table.read('/Users/Vince.ec/Downloads/z68_selected_in_clear_GN.txt', format = 'ascii').to_pandas()
S_cat = Table.read('/Users/Vince.ec/Downloads/z68_selected_in_clear_GS.txt', format = 'ascii').to_pandas()

In [3]:
unmatch_N_cat = N_cat.query('ID_3DHST < 0 or ID_3DHST > 500000')
unmatch_S_cat = S_cat.query('ID_3DHST < 0 or ID_3DHST > 500000')

In [4]:
unmatch_N_ids = unmatch_N_cat.ID_SF.values
unmatch_S_ids = unmatch_S_cat.ID_SF.values

In [5]:
GN_cat = Table.read('/Users/Vince.ec/Clear_data/goodsn_mosaic/goodsn-F105W-astrodrizzle-v4.4_drz_sub.cat', format = 'ascii').to_pandas()
GS_cat = Table.read('/Users/Vince.ec/Clear_data/goodss_mosaic/goodss-F105W-astrodrizzle-v4.4_drz_sub.cat', format = 'ascii').to_pandas()

In [7]:
KEYS = ['NUMBER', 'X_IMAGE', 'Y_IMAGE', 'CXX_IMAGE', 'CYY_IMAGE', 'CXY_IMAGE',
       'THETA_IMAGE', 'THETA_WORLD', 'ELLIPTICITY', 'KRON_RADIUS', 'A_IMAGE',
       'B_IMAGE', 'X_WORLD', 'Y_WORLD', 'BACKGROUND', 'FLUX_APER',
       'FLUX_APER_1', 'FLUX_APER_2', 'FLUX_APER_3', 'FLUX_APER_4',
       'FLUX_APER_5', 'FLUX_APER_6', 'FLUXERR_APER', 'FLUXERR_APER_1',
       'FLUXERR_APER_2', 'FLUXERR_APER_3', 'FLUXERR_APER_4', 'FLUXERR_APER_5',
       'FLUXERR_APER_6', 'MAG_APER', 'MAG_APER_1', 'MAG_APER_2', 'MAG_APER_3',
       'MAG_APER_4', 'MAG_APER_5', 'MAG_APER_6', 'MAGERR_APER',
       'MAGERR_APER_1', 'MAGERR_APER_2', 'MAGERR_APER_3', 'MAGERR_APER_4',
       'MAGERR_APER_5', 'MAGERR_APER_6', 'FLUX_AUTO', 'FLUXERR_AUTO',
       'MAG_AUTO', 'MAGERR_AUTO', 'FLUX_BEST', 'FLUXERR_BEST', 'MAG_BEST',
       'MAGERR_BEST', 'FLUX_RADIUS', 'ISOAREA_IMAGE', 'FWHM_IMAGE', 'FLAGS',
       'CLASS_STAR']

N_unmatch_dict = {}
S_unmatch_dict = {}

for k in KEYS:
    N_unmatch_dict[k] = []
    S_unmatch_dict[k] = []    

In [27]:
# segmap = fits.open('../data/GND_3DHST_plus_v2_seg.fits')

# w=wcs.WCS(segmap[0].header)

N_unmatch_dict = {}
for k in KEYS:
    N_unmatch_dict[k] = []

idx =0
for i in unmatch_N_cat.index:
    for k in KEYS:
        N_unmatch_dict[k].append(-99)
    N_unmatch_dict['NUMBER'][idx] = 5400000 + unmatch_N_cat.ID_SF[i]
    x,y = w.wcs_world2pix([[unmatch_N_cat.RA[i],unmatch_N_cat.Dec[i]]],1)[0]
    N_unmatch_dict['X_IMAGE'][idx] = x
    N_unmatch_dict['Y_IMAGE'][idx] = y
    idx += 1

In [33]:
segmap = fits.open('../data/GSD_3DHST_plus_v2_seg.fits')

w=wcs.WCS(segmap[0].header)

S_unmatch_dict = {}
for k in KEYS:
    S_unmatch_dict[k] = []

idx =0
for i in unmatch_S_cat.index:
    for k in KEYS:
        S_unmatch_dict[k].append(-99)
    S_unmatch_dict['NUMBER'][idx] = 5400000 + unmatch_S_cat.ID_SF[i]
    x,y = w.wcs_world2pix([[unmatch_S_cat.RA[i],unmatch_S_cat.Dec[i]]],1)[0]
    S_unmatch_dict['X_IMAGE'][idx] = x
    S_unmatch_dict['Y_IMAGE'][idx] = y
    idx += 1

In [34]:
import pandas as pd

update_Ncat  = pd.concat([GN_cat,pd.DataFrame(N_unmatch_dict)])[KEYS]
update_Scat  = pd.concat([GS_cat,pd.DataFrame(S_unmatch_dict)])[KEYS]

/Users/Vince.ec/miniconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Vince.ec/miniconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [36]:
update_Ncat.to_csv('../dataframes/galaxy_frames/GND_3DHST_plus_v2_seg.cat', sep=' ', index=False)
update_Scat.to_csv('../dataframes/galaxy_frames/GSD_3DHST_plus_v2_seg.cat', sep=' ', index=False)

In [64]:
Segmap_add(unmatch_N_ids,ID_modifier=54,segmap_with_sources='/Users/Vince.ec/Downloads/GOODSN_segmap.fits',
           segmap_to_adjust= '/Users/Vince.ec/Downloads/goodsn_3dhst.v4.0.F160W_seg.fits', file_out= '../data/GND_3DHST_plus_v2.fits')

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
29834
3 5
[[14059  9

In [82]:
Segmap_add(np.array(unmatch_S_ids).astype(int),54,'/Users/Vince.ec/Downloads/GOODSS_segmap.fits',
           '/Users/Vince.ec/Downloads/goodss_3dhst.v4.0.F160W_seg.fits','../data/GSD_3DHST_plus_v2.fits')

28651
2 3
[[ 8202 11146]
 [ 8205 11145]]
30868
4 4
[[ 8555 11587]
 [ 8560 11582]]
32457
5 5
[[ 9191 11868]
 [ 9196 11864]]
43046
5 4
[[ 8547 14136]
 [ 8550 14131]]
44043
16 14
[[ 8362 14397]
 [ 8376 14381]]
45164
11 12
[[10733 14654]
 [10745 14644]]
47386
14 16
[[11051 15208]
 [11067 15194]]
47398
29 17
[[11051 15256]
 [11068 15227]]
52335
9 11
[[14359 16296]
 [14370 16286]]
55619
11 19
[[13117 17091]
 [13136 17080]]
57677
25 47
[[ 9675 17787]
 [ 9721 17762]]
27001
3 3
[[ 8702 10793]
 [ 8704 10790]]
31395
9 9
[[ 8232 11690]
 [ 8241 11681]]
35288
28 21
[[ 8666 12455]
 [ 8686 12426]]
47463
7 5
[[ 7896 15222]
 [ 7901 15216]]
49216
35 23
[[14603 15689]
 [14627 15655]]
49653
16 27
[[14418 15734]
 [14445 15718]]
51770
15 10
[[ 8093 16182]
 [ 8102 16168]]
51997
8 5
[[ 8003 16223]
 [ 8007 16216]]
52029
15 14
[[ 7995 16230]
 [ 8009 16216]]
52228
7 7
[[ 9554 16267]
 [ 9561 16260]]
53128
9 10
[[ 9959 16478]
 [ 9968 16468]]
53131
8 6
[[13875 16476]
 [13882 16469]]
53196
38 26
[[13817 16577]
 [1384

ValueError: min() arg is an empty sequence

In [85]:
sources = unmatch_S_ids
ID_modifier = 54
segmap_with_sources= '/Users/Vince.ec/Downloads/GOODSS_segmap.fits'
segmap_to_adjust= '/Users/Vince.ec/Downloads/goodss_3dhst.v4.0.F160W_seg.fits'
file_out='../data/GSD_3DHST_plus_v2.fits' 
ID=sources

ID_modifier=ID_modifier*100000


###choose segmentation map and gather image data
segmap = fits.open(segmap_with_sources)
image = segmap[0].data

###choose which seg map to add info to
segmap_main = fits.open(segmap_to_adjust)
image_main = segmap_main[0].data

###get translation info to go from pix to wcs, or vice versa
w=wcs.WCS(segmap[0].header)
w_main=wcs.WCS(segmap_main[0].header)

In [86]:
r_min=[]
c_min=[]
r_max=[]
c_max=[]
###copy source into main file
for i in range(len(ID)):
    print(ID[i])
    indicies=np.argwhere(image==ID[i])  # get locations of source
    indicies=np.transpose(indicies)     # put into form that easier to organize
    r_min.append(min(indicies[0]))            # get indices to create smallest possible matrix of source
    c_min.append(min(indicies[1]))
    r_max.append(max(indicies[0])+1)
    c_max.append(max(indicies[1])+1)



28651
30868
32457
43046
44043
45164
47386
47398
52335
55619
57677
27001
31395
35288
47463
49216
49653
51770
51997
52029
52228
53128
53131
53196
53372
53382
53461
53489
53694
53726
53837
54128
54717
54799
57433
57820
58656
28927
32952
35116
54099
54792


In [87]:
for i in range(len(ID)):
    print(ID[i])
    source=image[r_min[i]:r_max[i],c_min[i]:c_max[i]]   # get source image
    source[source!=ID[i]]=0                 # delete any source that's not the one of interest
    source[source==ID[i]]=ID_modifier+ID[i]     # add unique id modifier to sources
    edges=w.wcs_pix2world([[c_min[i],r_max[i]],[c_max[i],r_min[i]]],1)  # get world coordinates of edges of source image
    area=w_main.wcs_world2pix(edges,1)      # find location where source will be added
    rowdiff=np.absolute(r_max[i]-r_min[i])        # make sure area in which the source will be added is the same size
    coldiff=np.absolute(c_max[i]-c_min[i])
    area = np.array(area).astype(int)
    print(rowdiff,coldiff)
    print(area)
    k=image_main[area[1][1]:area[1][1]+rowdiff,area[0][0]:area[0][0]+coldiff]
    for i in range(len(k)):                 # add source into segmentation map
        for ii in range(len(k[0])):
            if source[i][ii]!=0:
                k[i][ii]=source[i][ii]

fits.writeto(file_out,image_main,header=segmap_main[0].header)

28651
2 3
[[ 8202 11146]
 [ 8205 11145]]
30868
4 4
[[ 8555 11587]
 [ 8560 11582]]
32457
5 5
[[ 9191 11868]
 [ 9196 11864]]
43046
5 4
[[ 8547 14136]
 [ 8550 14131]]
44043
16 14
[[ 8362 14397]
 [ 8376 14381]]
45164
11 12
[[10733 14654]
 [10745 14644]]
47386
14 16
[[11051 15208]
 [11067 15194]]
47398
29 17
[[11051 15256]
 [11068 15227]]
52335
9 11
[[14359 16296]
 [14370 16286]]
55619
11 19
[[13117 17091]
 [13136 17080]]
57677
25 47
[[ 9675 17787]
 [ 9721 17762]]
27001
3 3
[[ 8702 10793]
 [ 8704 10790]]
31395
9 9
[[ 8232 11690]
 [ 8241 11681]]
35288
28 21
[[ 8666 12455]
 [ 8686 12426]]
47463
7 5
[[ 7896 15222]
 [ 7901 15216]]
49216
35 23
[[14603 15689]
 [14627 15655]]
49653
16 27
[[14418 15734]
 [14445 15718]]
51770
15 10
[[ 8093 16182]
 [ 8102 16168]]
51997
8 5
[[ 8003 16223]
 [ 8007 16216]]
52029
15 14
[[ 7995 16230]
 [ 8009 16216]]
52228
7 7
[[ 9554 16267]
 [ 9561 16260]]
53128
9 10
[[ 9959 16478]
 [ 9968 16468]]
53131
8 6
[[13875 16476]
 [13882 16469]]
53196
38 26
[[13817 16577]
 [1384

In [80]:
indicies

array([[27556, 27557, 27557, 27558, 27558, 27558, 27558, 27559, 27559,
        27559, 27559, 27559, 27560, 27560, 27560, 27560, 27560, 27560,
        27561, 27561, 27561, 27561, 27561, 27561, 27561, 27561, 27562,
        27562, 27562, 27562, 27562, 27562, 27562, 27562, 27562, 27563,
        27563, 27563, 27563, 27563, 27563, 27563, 27563, 27563, 27564,
        27564, 27564, 27564, 27564, 27564, 27564, 27564, 27564, 27564,
        27565, 27565, 27565, 27565, 27565, 27565, 27565, 27565, 27565,
        27566, 27566, 27566, 27566, 27566, 27566, 27566, 27566, 27566,
        27566, 27567, 27567, 27567, 27567, 27567, 27567, 27567, 27567,
        27567, 27568, 27568, 27568, 27568, 27568, 27568, 27568, 27568,
        27568, 27569, 27569, 27569],
       [18826, 18826, 18827, 18825, 18826, 18827, 18828, 18825, 18826,
        18827, 18828, 18829, 18824, 18825, 18826, 18827, 18828, 18829,
        18823, 18824, 18825, 18826, 18827, 18828, 18829, 18830, 18823,
        18824, 18825, 18826, 18827, 1882

In [73]:
np.max(image)

60032

In [78]:
S_cat.query('ID_SF == 53461')

,ID_SF,RA,Dec,z_peak,ID_3DHST,RA_3DHST,Dec_3DHST
73,53461,53.056369,-27.705793,6.739701,-1,-1.0,-1.0


In [81]:
unmatch_S_ids

array([28651, 30868, 32457, 43046, 44043, 45164, 47386, 47398, 52335,
       55619, 57677, 27001, 31395, 35288, 47463, 49216, 49653, 51770,
       51997, 52029, 52228, 53128, 53131, 53196, 53372, 53382, 53461,
       53489, 53694, 53726, 53837, 54128, 54717, 54799, 57433, 57820,
       58656, 28927, 32952, 35116, 54099, 54792])